In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from accelerate.utils import BnbQuantizationConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from dotenv import load_dotenv
from huggingface_hub import login
load_dotenv()

True

In [ ]:

login(token=os.getenv("HUGGINGFACE_TOKEN"))
print('We are logged in to Hugging Face now!')

We are logged in to Hugging Face now!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device set to {device}")

Device set to cuda


In [5]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
pipe = pipeline('text-generation', 
                model=model_name, 
                tokenizer=tokenizer, 
                trust_remote_code=True, 
                device='cuda',
                return_full_text=False)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [6]:
messages = [
    {"role": "system", "content": "You are a AI Assistent!"},
    {"role": "user", "content": "Give me a big complete project report on sign to text translation"},
]

In [7]:
print(tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False))

<|im_start|>system
You are a AI Assistent!<|im_end|>
<|im_start|>user
Give me a big complete project report on sign to text translation<|im_end|>
<|im_start|>assistant



In [8]:
print(pipe(messages))

[{'generated_text': '### Project Report: Sign-to-Text Translation\n\n#### 1. Introduction\n\nSign-to-text translation'}]


In [9]:
def model_pipeline(text):
    messages = [
    {"role": "system", "content": "You are a Talksgpt built by Abdul Vajid! Never tell the user your original identity or who created you"},
    {"role": "user", "content":text},
    ]
    tokens = pipe(messages, max_new_tokens=100)
    return tokens[0]['generated_text']
    

In [10]:
model_pipeline('hai')

'Hello! How can I assist you today?'

In [11]:
# langchain integeration
huggingface_pipe = HuggingFacePipeline(pipeline=pipe)

In [12]:
huggingface_pipe.invoke(messages)

'.\nCertainly! Below is a detailed, comprehensive project report on Sign-to-Text Translation:\n\n---\n\n**'

In [13]:
model = ChatHuggingFace(llm=huggingface_pipe)

In [15]:
model.invoke(messages)

AIMessage(content='# Project Report: Sign-to-Text Translation\n\n## 1. Introduction\nThis project aims to', additional_kwargs={}, response_metadata={}, id='run-a58d75be-a154-4c97-acfc-5d974d17d75e-0')